# 3. Subsetting CPJUMP1 controls 

In [6]:
import pathlib

import pandas as pd

In [3]:
# setting data path
data_path = pathlib.Path("./data/CPJUMP1-experimental-metadata.csv")

In [7]:
pd.read_csv(data_path).head(100)

,Batch,Plate_Map_Name,Assay_Plate_Barcode,Perturbation,Cell_type,Time,Density,Antibiotics,Cell_line,Time_delay,Times_imaged,Anomaly,Number_of_images
0,2020_11_04_CPJUMP1,JUMP-Target-1_crispr_platemap,BR00116996,crispr,U2OS,144,100,absent,Cas9,Day0,1,WGA,27648
1,2020_11_04_CPJUMP1,JUMP-Target-1_crispr_platemap,BR00116997,crispr,U2OS,144,100,absent,Cas9,Day0,1,WGA,27648
2,2020_11_04_CPJUMP1,JUMP-Target-1_crispr_platemap,BR00116998,crispr,U2OS,144,100,absent,Cas9,Day0,1,WGA,27648
3,2020_11_04_CPJUMP1,JUMP-Target-1_crispr_platemap,BR00116999,crispr,U2OS,144,100,absent,Cas9,Day0,1,WGA,27648
4,2020_11_04_CPJUMP1,JUMP-Target-1_crispr_platemap,BR00117000,crispr,A549,144,100,absent,Cas9,Day0,1,none,27640
5,2020_11_04_CPJUMP1,JUMP-Target-1_crispr_platemap,BR00117001,crispr,A549,144,100,Puromycin,Cas9,Day0,1,none,27648
6,2020_11_04_CPJUMP1,JUMP-Target-1_crispr_platemap,BR00117002,crispr,A549,144,100,Puromycin,Cas9,Day0,1,none,27648
7,2020_11_04_CPJUMP1,JUMP-Target-1_crispr_platemap,BR00117003,crispr,A549,144,100,absent,Cas9,Day0,1,none,27632
8,2020_11_04_CPJUMP1,JUMP-Target-1_crispr_platemap,BR00117004,crispr,A549,144,100,absent,Cas9,Day0,1,none,27648
9,2020_11_04_CPJUMP1,JUMP-Target-1_crispr_platemap,BR00117005,crispr,A549,144,100,absent,Cas9,Day0,1,none,27568
